In [13]:
import cv2
import numpy as np
from scipy.optimize import minimize

In [14]:
degraded_image = cv2.imread('Lincoln.jpg', cv2.IMREAD_GRAYSCALE)

In [15]:
PATCH_SIZE = 64  # Adjust as needed

In [16]:
def total_variation(image):
    dx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    tv = np.sum(np.sqrt(dx**2 + dy**2))
    return tv

In [17]:
def objective_func(x, patch, alpha):
    x = x.reshape(patch.shape)
    data_fidelity = np.sum((x - patch)**2)
    return data_fidelity + alpha * total_variation(x)

In [18]:
def restore_image_tv(degraded_image, alpha, max_iter=20):
    height, width = degraded_image.shape[:2]
    restored = np.zeros_like(degraded_image)

    for x in range(0, width, PATCH_SIZE):
        for y in range(0, height, PATCH_SIZE):
            patch = degraded_image[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            initial_guess = np.copy(patch)
            result = minimize(objective_func, initial_guess.flatten(), args=(patch, alpha),
                              bounds=[(0, 255)] * patch.size, method='L-BFGS-B', options={'maxiter': max_iter})
            restored_patch = result.x.reshape(patch.shape)
            restored[y:y+PATCH_SIZE, x:x+PATCH_SIZE] = restored_patch

    restored = np.clip(restored, 0, 255).astype(np.uint8)
    return restored

In [19]:
restored = restore_image_tv(degraded_image, alpha=0.1)

In [20]:
cv2.imwrite('restored_image.jpg', restored)
cv2.imshow('Restored Image', restored)
cv2.waitKey(0)
cv2.destroyAllWindows()